In [ ]:
import numpy as np
import pandas as pd
import nltk
from nltk.chat.util import Chat,reflections
import string
import random
from nltk.stem import PorterStemmer, WordNetLemmatizer
import spacy

In [ ]:
df_symptom=pd.read_csv('Desktop\data_symp.csv')
df_symptom.columns = df_symptom.columns.str.lower()
df_symptom.columns
df_symptom.drop("unnamed: 0",axis=1,inplace=True)

In [ ]:
Chat
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [ ]:
nlp= spacy.load('en_core_web_lg')

In [ ]:
our_diseases=['covid', 'Heart_stroke', 'uti', 'Allergy', 'Cold', 'Flu',
       'magraine', 'Panic Disorder', 'pharyngitis',
       'crise_cardiaque']

In [ ]:
import joblib

In [ ]:
model = joblib.load('model_.joblib')

In [ ]:
all_symptoms

['gender',
 'age',
 'fever',
 'cough',
 'runny_nose',
 'muscle_ache',
 'pneumonia',
 'diarrhea',
 'lung_infection',
 'travel_history',
 'isolation_treatment',
 'currentsmoker',
 'cigsperday',
 'bpmeds',
 'prevalentstroke',
 'prevalenthyp',
 'diabetes',
 'sysbp',
 'diabp',
 'bmi',
 'heartrate',
 'nausea',
 'lumbar_pain',
 'urine_pushing',
 'micturition_pains',
 'burning_urethra',
 'inflamation',
 'nephritis',
 'tiredness',
 'sore_throat',
 'stuffy_nose',
 'vomit',
 'shortness_of_breath',
 'difficulty_breathing',
 'loss_of_taste',
 'loss_of_smell',
 'itchy_nose',
 'itchy_eyes',
 'itchy_mouth',
 'itchy_inner_ear',
 'sneezing',
 'pink_eye',
 'duration',
 'frequency',
 'location',
 'character',
 'intensity',
 'phonophobia',
 'photophobia',
 'visual',
 'sensory',
 'dysphasia',
 'dysarthria',
 'vertigo',
 'tinnitus',
 'dpf',
 'chest pain',
 'dizziness',
 'fear of losing control',
 'panic attacks',
 'pain',
 'headache',
 'tonsillarswelling',
 'exudate',
 'sudden',
 'chest_pain',
 'trestbps',
 

In [ ]:
all_symptoms = df_symptom.drop('target',axis=1).columns.tolist()
def process_text(text):
    text = text.lower()
    doc = nlp(text)
    processed_words = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct]
    return processed_words
def same_length(liste1, liste2):
    common_elements = set(liste1) & set(liste2)

    list1_same_length = [item if item in common_elements else 0 for item in liste2]
    return list1_same_length
def convert_to_symptom_vector(symptoms,all_symptoms):
    vector=[]
    symptoms=same_length(symptoms,all_symptoms)
    vector_df = pd.DataFrame(columns=all_symptoms)
    for symptom in symptoms:
        if symptom in all_symptoms:
            vector.append(1)
        else:
            vector.append(0)

    vector_df = pd.concat([vector_df, pd.DataFrame([vector], columns=all_symptoms)], ignore_index=True)
    if symptoms[0]=="male":
        vector_df['gender']=0
    elif symptoms[0]=="female" :
        vector_df['gender']=1
    vector_df['age']=symptoms[1]
    vector_df = vector_df.astype(int)
    return vector_df
def extract_symptoms_from_input(user_inputs,all_symptoms):
    processed_input=process_text(user_inputs)
    identified_symptoms=[token for token in processed_input if token in all_symptoms[2:]]
    return identified_symptoms
def predict_diseases(symptom_vector):
    our_disease=['Allergy', 'Cold', 'Flu', 'Heart_stroke', 'Panic Disorder',
       'covid', 'crise_cardiaque', 'magraine', 'pharyngitis', 'uti']
    predicted_disease=model.predict(symptom_vector)

    return our_disease[int(predicted_disease)]

In [ ]:
symptom_vector=convert_to_symptom_vector(symptoms,all_symptoms)
symptom_vector.dtypes
predict_diseases(symptom_vector)

'covid'

In [ ]:
def ask_for_gender():
    while True:
        gender = input("Chatbot: What is your gender? \n").strip().lower()
        if gender in ['male', 'female']:
            return gender
        else:
            print("Chatbot: Please enter a valid gender ('male' or 'female'). \n")

def ask_for_age():
    while True:
        try:
            age = int(input("Chatbot: Please provide your age: \n"))
            if age > 0:
                return str(age)
            else:
                print("Chatbot: Please enter a valid age (greater than 0). \n")
        except ValueError:
            print("Chatbot: Please enter a valid age (numeric value). \n")

In [ ]:
while True:

    user_inputs = []
    print("Chatbot: Hi! How can I help you today? \n")
    hi_input = input("You: \n")
    print("Chatbot:I hope that I can help you \n")
    user_gender = ask_for_gender()
    user_age = ask_for_age()
    user_inputs.extend([user_gender,user_age])
    print("Chatbot: Okay, now I need to know your symptoms. Please enter them below.  \n")
    user_symptoms=input("You: ")
    if user_symptoms.lower() in ['exit', 'quit', 'bye']:
        print("Chatbot: Goodbye!")
        break
    else:
        user_inputs.append(user_symptoms)

    print(user_inputs)
    identified_symptoms = extract_symptoms_from_input(' '.join(user_inputs), all_symptoms)

    if identified_symptoms:
    # Convert identified symptoms to a binary vector DataFrame
        symptom_vector = convert_to_symptom_vector(identified_symptoms, all_symptoms)

    # Call your disease detection model to get the predicted disease
        predicted_disease = predict_diseases(symptom_vector)

        if predicted_disease:
            print("Chatbot: Based on your symptoms, you might have", predicted_disease + ". Please consult a doctor for proper diagnosis and treatment.")
        else:
            print("Chatbot: I'm sorry, I couldn't predict any specific disease based on the provided symptoms.  \n")
    else:
        print("Chatbot: I couldn't identify any specific symptoms. Can you please provide more details?  \n")

Chatbot: Hi! How can I help you today? 

You: 
rehdhhsscijdhis
Chatbot:I hope that I can help you 

Chatbot: What is your gender? 
male 
Chatbot: Please provide your age: 
22
Chatbot: Okay, now I need to know your symptoms. Please enter them below.  

You: fever and cough 
['male', '22', 'fever and cough ']
Chatbot: Based on your symptoms, you might have covid. Please consult a doctor for proper diagnosis and treatment.
Chatbot: Hi! How can I help you today? 

